# First I will start with the entire Data pipeline (with documentation), and them my queries will be at the bottom.

**Spin up the cluster:**
    
   
```python
docker-compose up -d    
```

**Run flask. Remember to call correct port number, and run the game API. This has the routes for both sword/guild. Go to the Game API python file to see my logic.**

  ```python
docker-compose exec mids \
  env FLASK_APP=/w205/project-3-honlineh/game_api.py \
  flask run --host 0.0.0.0
```
  
Everytime an event is called, output looks as such. The 200 means it is a succesful event:

127.0.0.1 - - [11/Dec/2019 22:30:14] "GET /purchase_a_sword HTTP/1.0" 200 -

**Set up Kafka Cat to watch for events:**
    
```python
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning
    ```

The output looks as such:

{"event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "discount": "No", "Host": "user1.comcast.com","description": "Bronze"}
{"event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "discount": "No", "Host": "user1.comcast.com","description": "Bronze"}


**In order to get the two types of output above, we need to run some Apache Bench test data. This allows us to to generate data as if someone was playing the game.**

To do so, we run:

```python
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword
```

**Next, we peek into Hadoop to make sure that our sword purchase data is being stored:**
    
```python
docker-compose exec cloudera hadoop fs -ls /tmp/purchases/
```

**Let us start with Purchasing a Sword. We have to run the "Write_Swords_Stream" file. I have commented that file to mention what each parts does. Go to that file now. We use the following command to do so. Note we are NOT using Batch, we are only using Stream.**

```python
docker-compose exec spark spark-submit /w205/project-3-honlineh/write_swords_stream.py
```

**Now, we go into Hive. Hive allows us to a create a table we can later query off, and allows us to keep track of various Schema we may need. I make sure to specify the Schema in the way I want it (with discount and sword type)**

```python
docker-compose exec cloudera hive
```

**Note how I add description and discount at the end**

```python
create external table if not exists default.sword_purchases (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    description string,
    discount string
  )
  stored as parquet 
  location '/tmp/sword_purchases'
  tblproperties ("parquet.compress"="SNAPPY");
```


**Now, continously create data using Apache Bench every 10 seconds in an infinite while loop. I will use this to conduct some analytics later on in the report.**

```python

while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/purchase_a_sword
  sleep 10
done
```

**I then fire up Presto for querying capabilities**

```python
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default
```

**Within Presto, I can see which tables I have. At this point, I have created both the guild and sword table. Output is below.**

```python
show tables;
```
**As a result, we get:**
```
presto:default> show tables;
      Table
-----------------+
 join_guild
 sword_purchases
```

**We can also go into the schema of the swords table using:**
    
```
describe sword_purchases;
```

**From this, we get:**
    
```
   Column    |  Type   | Comment
-------------+---------+---------
 raw_event   | varchar |
 timestamp   | varchar |
 accept      | varchar |
 host        | varchar |
 user_agent  | varchar |
 event_type  | varchar |
 description | varchar |
 discount    | varchar |
 ```

**Before doing any actual querying that will give us some insights, we do some EDA. Specifically, I want to make sure that all the 8 fields have the proper values, as I did change the schema. Thus, we do the following:**

```
select Host from sword_purchases;

select timestamp from sword_purchases;

select * from sword_purchases;
```

Great, everything looks good! Table is populating nicely.

##  Querying/Business Recommendations

```
presto:default> select count(*) from sword_purchases;
 _col0
-------+
    20
(1 row)

##after 120 seconds

presto:default> select count(*) from sword_purchases;
 _col0
-------+
   120
(1 row)

##after some more time

presto:default> select count(*) from sword_purchases;
 _col0
-------+
   720
(1 row)
```
I keep it growing until I hit 2020 entries. This is perhaphs the most vital part of the entire pipeline--because I see data being populated, and because my EDA checked out, I know my pipeline is giving me the desired results.

**We then run a couple queries on this table of ouput. As game creators, we have a rate of 1/5 for getting silver swords, and 1/25 of getting gold swords. We run a query to confirm this:**

```
presto:default> Select description, count(*) from sword_purchases group by description;
 description | _col1
-------------+-------
 Bronze      |   1621
 Silver      |    315
 Gold        |     84
 ```
 
 Ultimately, we get 80% bronze, 16% Silver, and 4% Gold. From a business perspective, we can offer a higher amount of silver/gold values if the player pays for it.

**We also offer a discount for certain members, but this would be extremely lucky. A person has in a 1/100 chance of getting a discount on the sword. On the current dataset, we have the following distribution:**

```
presto:default> Select discount, count(*) from sword_purchases group by discount;
 discount | _col1
----------+-------
 No       |   2003
 Yes      |     17
 ```
 
Here, we see the value of a discount to be 0.8%, which is below our expected 1%. We want to make sure there are not too many discounts!

**We then look at the lucky participants who get BOTH a gold sword and a discount on it. What a steal!! We as game developers need to make sure this doesn't happen often:**

```
presto:default> Select count(*) from sword_purchases where description = 'Gold' AND discount = 'Yes';
 _col1  
----------+
 3          
```

Indeed it only happens 3/2020, or 0.1485% of the time. If this happens too many times, the game would be "broken". From an exepcted value perspective, this should happen (1% * 4%) of the time - 0.04%. Though our sample size isn't the biggest, we are higher than where we expect to be.

**We can then run similar pieces of analysis on Join guild. This is ommitted due to repetition. I am including the similar queries to the ones I ran above for Swords:**

Make sure joining guild produces proper response
```python
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_guild
```
Running streaming file

```python
docker-compose exec spark spark-submit /w205/project-3-honlineh/write_guild_stream.py
```

The output looks like: 
{"Accept": "*/*", "Host": "user1.comcast.com", "event_type": "join_guild", "guild_type": "2", "User-Agent": "ApacheBench/2.3"}
{"Accept": "*/*", "Host": "user1.comcast.com", "event_type": "join_guild", "guild_type": "1", "User-Agent": "ApacheBench/2.3"}

Creating Hive table
```python
create external table if not exists default.join_guild (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    guild_type string
  )
  stored as parquet 
  location '/tmp/join_guild'
  tblproperties ("parquet.compress"="SNAPPY");
``` 
Using presto to outlay schema 
```python
describe join_guild;
``` 
Seeing all the requests

```python
select * from join_guild;
```

Continously pushing additional data
```python
while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/join_guild
  sleep 10
done
```




**Finally, we shut down docker.**
```
docker-compose down
```